In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv('train.csv')
test_data  = pd.read_csv('test.csv')

In [3]:
train_data.head()

,id,Label,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
0,TCGA-2V-A95S-01A,-1,1.322600,0.000000,2.60713,0,1.411440,3.27352,0,0.006410,...,0,2.710970,0.0,0.435580,0.000000,3.90541,0.017601,0,1.708020,0.0
1,TCGA-2Y-A9GU-01A,1,0.000000,0.006431,1.49737,0,0.598584,2.39426,0,0.006599,...,0,0.812853,0.0,0.208534,0.147081,4.05334,0.368102,0,0.182464,0.0
2,TCGA-2Y-A9GV-01A,1,0.000000,0.000000,1.28544,0,1.366550,2.36780,0,0.005341,...,0,6.645780,0.0,0.170978,0.176644,2.90339,0.262281,0,0.386022,0.0
3,TCGA-2Y-A9GX-01A,1,0.201191,0.000000,1.45312,0,1.566180,2.84619,0,0.006135,...,0,5.085340,0.0,0.067894,0.000000,3.70349,0.422939,0,0.666707,0.0
4,TCGA-2Y-A9GY-01A,-1,0.043040,0.000000,2.25014,0,1.421440,2.73472,0,0.024669,...,0,0.140969,0.0,0.370195,0.000000,3.85624,0.236684,0,0.513950,0.0


In [4]:
train_data.shape

(279, 60485)

In [5]:
X = train_data.drop(columns=['id', 'Label'])
y = train_data.iloc[:, 1]

In [6]:
y.head()

0   -1
1    1
2    1
3    1
4   -1
Name: Label, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
# split into 80:20 ration
X_train, X_CV, y_train, y_CV = train_test_split(X, y, test_size = 0.2)

In [8]:
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest

In [9]:
# Create an SelectKBest object to select features with two best ANOVA F-Values
fvalue_selector = SelectKBest(f_classif, k=100)

# Apply the SelectKBest object to the features and target
X_kbest = fvalue_selector.fit_transform(X_train, y_train)
X_kbest_CV = fvalue_selector.transform(X_CV)

C:\Users\Ridam\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [    3     6    12 ... 60462 60473 60480] are constant.
  UserWarning)
C:\Users\Ridam\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [10]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

In [11]:
clf.fit(X_kbest, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [12]:
clf.score(X_kbest, y_train)

1.0

In [13]:
clf.score(X_kbest_CV, y_CV)

0.5357142857142857

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
parameter_candidates = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['poly']}
]

In [17]:
# Create a classifier object with the classifier and parameter candidates
clf = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=parameter_candidates, cv=10, n_jobs=-1)

# Train the classifier on data1's feature and target data
clf.fit(X_kbest, y_train) 

ValueError: Invalid parameter C for estimator DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best'). Check the list of available parameters with `estimator.get_params().keys()`.

In [20]:
# View the accuracy score
print('Best score for data1:', clf.best_score_) 
print('Best score for data1:', clf.best_params_) 

Best score for data1: 0.695067264573991
Best score for data1: {'C': 1, 'gamma': 0.001, 'kernel': 'poly'}


In [21]:
clf = SVC(C= 10, kernel = 'rbf', gamma = 0.001,probability = True)
clf.fit(X_kbest, y_train) 
clf.score(X_kbest, y_train) 

0.7713004484304933

In [22]:
clf.score(X_kbest_CV,y_CV)

0.6428571428571429

In [54]:
X1 = test_data.drop(columns=['id'])

In [55]:
X1.head()

,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,ENSG00000259883.1,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
0,0.000000,0.006764,1.68576,0.0,1.046760,2.56965,0,0.000000,5.38054,0.000000,...,0,6.30029,0,0.076595,0.000000,3.02069,0.196880,0,0.538068,0.0
1,0.247055,0.000000,1.89639,0.0,1.996530,2.47638,0,0.016952,5.98824,0.228977,...,0,5.04563,0,0.137816,0.000000,3.56454,0.397989,0,0.574163,0.0
2,0.026951,0.000000,1.80290,0.0,1.568690,1.90269,0,0.007726,5.97967,0.096880,...,0,6.00935,0,0.000000,0.044642,3.64769,0.217624,0,0.135791,0.0
3,0.000000,0.000000,1.51205,0.0,0.938359,1.81372,0,0.003923,6.46130,0.000000,...,0,5.51651,0,0.127467,0.000000,2.88242,0.612775,0,0.807993,0.0
4,0.643548,0.000000,1.35997,0.0,1.254810,3.64841,0,0.128797,6.27939,0.033555,...,0,2.28184,0,0.058164,0.060170,4.11811,0.150976,0,0.416153,0.0


In [56]:
X_id = test_data['id']

In [57]:
type(X_id)

pandas.core.series.Series

In [58]:
X_id.head()

0    3001
1    3002
2    3003
3    3004
4    3005
Name: id, dtype: int64

In [127]:
# Create an SelectKBest object to select features with two best ANOVA F-Values
fvalue_selector = SelectKBest(f_classif, k=100)

# Apply the SelectKBest object to the features and target
X_kbest = fvalue_selector.fit_transform(X, y)
X_kbest_test = fvalue_selector.transform(X1)

C:\Users\Ridam\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [    3     6    12 ... 60462 60473 60480] are constant.
  UserWarning)
C:\Users\Ridam\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [128]:
clf = SVC(C= 10, kernel = 'rbf', gamma = 0.001)
clf.fit(X_kbest, y)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [129]:
clf.score(X_kbest, y)

0.8064516129032258

In [131]:
y_predict = clf.predict(X_kbest_test)

In [132]:
new_series = pd.DataFrame(y_predict)

In [133]:
new_series1 = pd.DataFrame(X_id)

In [134]:
type(new_series)

pandas.core.frame.DataFrame

In [135]:
sample = new_series1.join(new_series)

In [136]:
type(sample)

pandas.core.frame.DataFrame

In [137]:
sample.columns = ['id','Label']

In [138]:
sample.shape

(71, 2)

In [140]:
sample.to_csv('sample.csv',index = False) 